In [1]:
import numpy as np

from scipy.constants import e, c

from scipy.constants import physical_constants

In [2]:
from cpymad.madx import Madx

import sixtracklib as stl
import pysixtrack

# Versioning

## SixTrackLib

In [3]:
stl.__file__

'/home/oeftiger/gsi/git/sixtracklib/python/sixtracklib/__init__.py'

In [4]:
!cd '/home/oeftiger/gsi/git/sixtracklib/python/sixtracklib/' && git log | head -4

commit 0539914c228a66820427b2c7d333725fcc02ebd7
Merge: 00d1a3a7 af5af8ea
Author: Martin Schwinzerl <martin.schwinzerl@cern.ch>
Date:   Mon Dec 16 19:05:33 2019 +0100


## PySixTrack

In [5]:
pysixtrack.__file__

'/home/oeftiger/gsi/git/pysixtrack/pysixtrack/__init__.py'

In [6]:
!cd '/home/oeftiger/gsi/git/pysixtrack/pysixtrack/' && git log | head -4

commit 3429831a7a2a578213f8bb7f1de3792715dad2bd
Author: Riccardo De Maria <riccardodemaria@gmail.com>
Date:   Wed Dec 18 18:04:48 2019 +0100



# Parameters

In [7]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3
# nmass = 0.931494061 # MAD-X value

In [8]:
A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)

## MAD-X setup

In [9]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.01.03 11:51:59      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [10]:
madx.input('''SET, format="22.14e";''')

True

In [11]:
madx.input('''
s: Marker;

drifting: sequence, l = 1;
s, at = 1;
endsequence;
''')

True

In [12]:
madx.command.beam(
    particle='ion', mass=A*nmass, charge=Q, energy=Etot)

True

In [13]:
madx.use(sequence='drifting')

In [14]:
madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.25000000000000e-01   0.00000000000000e+00   2.00000000000000e+00   0.00000000000000e+00 

                 dxrms                 xcomax                 xcorms                     q2 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   1.25000000000000e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   2.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                ycomax                 ycorms                 deltap                synch_1 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

## MAD-X tracking

In [15]:
madx.input('''
TRACK, onepass, onetable, file=output.;

START, x=1e-03, px=1e-06, y=-1e-03, py=-5e-07, t=0.1, pt=3e-03;
!START, x=0e-03, px=0e-06, y=0e-03, py=0e-07, t=0.1, pt=3e-03;

RUN, turns=1;

ENDTRACK;
''')

enter TRACK module
one pass is on

++++++ table: tracksumm

    number       turn                      x                     px 
         1          0   1.00000000000000e-03   1.00000000000000e-06 
         1          1   1.00099475251863e-03   1.00000000000000e-06 

                     y                     py                      t                     pt 
 -1.00000000000000e-03  -5.00000000000000e-07   1.00000000000000e-01   2.99999999999989e-03 
 -1.00049737625931e-03  -5.00000000000000e-07   1.06259314454102e-01   2.99999999999989e-03 

                     s                      e 
  0.00000000000000e+00   0.00000000000000e+00 
  1.00000000000000e+00   0.00000000000000e+00 
exit TRACK module



True

In [16]:
headers = list(np.genfromtxt(
    "output.one", skip_header=51, max_rows=1, dtype=str)[1:])

initial_distribution_madx = np.genfromtxt(
    "output.one", skip_header=54, max_rows=1, dtype=np.float64)

final_distribution_madx = np.genfromtxt(
    "output.one", skip_header=55, max_rows=1, dtype=np.float64)

In [17]:
def get_betai(PT, mass=mass, p0=p0):
    restmass = mass * c**2
    restmass_sq = restmass**2
    E0 = np.sqrt((p0 * c)**2 + restmass_sq)
    
    E = E0 + PT * p0 * c
    gammai = E / restmass
    betai = np.sqrt(1 - 1. / (gammai * gammai))
    return betai

In [18]:
betai = get_betai(final_distribution_madx[headers.index('PT')])

x_madx = final_distribution_madx[headers.index('X')]
px_madx = final_distribution_madx[headers.index('PX')]
y_madx = final_distribution_madx[headers.index('Y')]
py_madx = final_distribution_madx[headers.index('PY')]
zeta_madx = final_distribution_madx[headers.index('T')] * betai
delta_madx = final_distribution_madx[headers.index('PT')] / betai

## SixTrackLib setup

In [19]:
# PySixTrack, lattice transfer and preparation!

pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.drifting, exact_drift=True,
)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

In [20]:
elements = stl.Elements.from_line(pysixtrack_elements)
elements.BeamMonitor(num_stores=1);

In [21]:
pysixtrack_particles = pysixtrack.Particles.from_madx_track(madx)

In [22]:
# necessary to transfer to SixTrackLib with this version of PySixTrack
pysixtrack_particles.state = np.array([1, 0])
pysixtrack_particles.elemid = 0

pysixtrack_particles.remove_lost_particles()

In [24]:
particles = stl.ParticlesSet().Particles(num_particles=1)

particles.from_pysixtrack(pysixtrack_particles, particle_index=0)

In [25]:
trackjob = stl.TrackJob(elements, particles, device="opencl:0.0")

## SixTrackLib tracking

In [26]:
trackjob.track_until(1)
trackjob.collect()

In [27]:
x = trackjob.output.particles[0].x[0]
px = trackjob.output.particles[0].px[0]
y = trackjob.output.particles[0].y[0]
py = trackjob.output.particles[0].py[0]
zeta = trackjob.output.particles[0].zeta[0]
delta = trackjob.output.particles[0].delta[0]

# evaluation SixTrackLib vs. MAD-X:

In [28]:
print ('x error in %: {}'.format(100 * (x - x_madx) / x))

x error in %: -3.24937419429278e-13


In [29]:
print ('y error in %: {}'.format(100 * (y - y_madx) / y))

y error in %: 3.250989552433857e-13


In [30]:
print ('zeta error in %: {}'.format(100 * (zeta - zeta_madx) / zeta))

zeta error in %: 1.14621532989312e-14
